In [1]:
!pip install ultralytics

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import drive
from google.colab.patches import cv2_imshow

# ------------------------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------------------------
drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/HITAI/kabbadi/models/best_yolov8_kabbadi.pt'
video_path = '/content/drive/MyDrive/HITAI/kabbadi/datasetv2/video_test.mp4'
output_video_path = '/content/drive/MyDrive/HITAI/kabbadi/datasetv2/output_with_map.mp4'

# Corner points (detected once)
corner_points = {3: None, 4: None, 5: None, 6: None}
center_point = None

# 2D map size
map_width = 650
map_height = 500

# Position in final video where we overlay the 2D map
overlay_x = 720
overlay_y = 750
overlay_map_size = (520, 400)

# Create model
model = YOLO(model_path)

# Video capture and writer
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_w, frame_h))

def draw_kabaddi_court(base_width, base_height):
    img = np.zeros((base_height, base_width, 3), dtype='uint8')
    img[:] = (255, 191, 0)
    margin = 10
    cv2.rectangle(img, (margin, margin), (base_width - margin, base_height - margin), (255, 255, 255), 2)
    top_rect_height = 40
    cv2.rectangle(img, (margin+2, margin+2), (base_width - margin-2, margin+top_rect_height), (0, 128, 255), -1)
    bottom_rect_height = 40
    cv2.rectangle(img, (margin+2, base_height - margin - 2 - bottom_rect_height),
                  (base_width - margin - 2, base_height - margin - 2), (0, 128, 255), -1)
    cv2.rectangle(img, (margin+2, margin + top_rect_height + 4),
                  (base_width - margin - 2, base_height - margin - bottom_rect_height - 4), (255, 0, 255), -1)
    center_line_y = base_height // 2
    cv2.line(img, (base_width//2, margin+2), (base_width//2, base_height - margin - 2), (255, 255, 255), 2)
    offset = int(base_width * 0.21)
    cv2.line(img, (offset, margin+2), (offset, base_height - margin - 2), (255, 255, 255), 2)
    cv2.line(img, (base_width - offset, margin+2), (base_width - offset, base_height - margin - 2), (255, 255, 255), 2)
    return img

kabaddi_map = draw_kabaddi_court(map_width, map_height)
homography_matrix = None
homography_computed = False

# Store player trails
player_trails = {0: [], 1: []}
player_last_seen = {0: 0, 1: 0}
trail_length = 800  # Maximum number of trail points
max_inactive_frames = 10  # Frames to wait before clearing trails

frame_no = 0
# Frame classification buffer for smoothing
classification_buffer = {0: [], 1: []}  # Buffer for Team 0 and Team 1
buffer_size = 3  # Use a sliding window of 3 frames (previous, current, next)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_no += 1

    results = model.predict(frame, conf=0.3)
    current_players = {0: False, 1: False}

    if not homography_computed:
        for result in results:
            boxes = result.boxes.xyxy
            classes = result.boxes.cls
            for box, cls_id in zip(boxes, classes):
                cls_id = int(cls_id.item())
                x1, y1, x2, y2 = map(int, box[:4])
                cx = (x1 + x2) // 2
                cy = y2

                if cls_id in corner_points.keys():
                    corner_points[cls_id] = (cx, cy)
                elif cls_id == 2:
                    center_point = (cx, cy)

        if all(corner_points[k] is not None for k in corner_points):
            pts_src = np.array([corner_points[3], corner_points[4], corner_points[5], corner_points[6]], dtype='float32')
            pts_dst = np.array([[0, 0], [map_width, 0], [0, map_height], [map_width, map_height]], dtype='float32')
            homography_matrix, status = cv2.findHomography(pts_src, pts_dst, cv2.RANSAC, 5.0)
            homography_computed = True

    overlay_map = kabaddi_map.copy()

    if homography_computed:
        frame_classifications = {0: [], 1: []}

        for result in results:
            boxes = result.boxes.xyxy
            classes = result.boxes.cls
            for box, cls_id in zip(boxes, classes):
                cls_id = int(cls_id.item())
                x1, y1, x2, y2 = map(int, box[:4])
                cx = (x1 + x2) // 2
                cy = y2

                if cls_id in [0, 1]:
                    frame_classifications[cls_id].append((cx, cy))
                    current_players[cls_id] = True
                    player_last_seen[cls_id] = frame_no
                    pt_src = np.array([[[cx, cy]]], dtype='float32')
                    pt_dst = cv2.perspectiveTransform(pt_src, homography_matrix)
                    px, py = pt_dst[0][0]

                    player_trails[cls_id].append((int(px), int(py)))
                    if len(player_trails[cls_id]) > trail_length:
                        player_trails[cls_id].pop(0)

                    dot_color = (0, 255, 0) if cls_id == 0 else (255, 0, 0)
                    cv2.circle(overlay_map, (int(px), int(py)), 15, dot_color, -1)

        # Add current frame classifications to the buffer
        for cls_id in [0, 1]:
            classification_buffer[cls_id].append(len(frame_classifications[cls_id]))
            if len(classification_buffer[cls_id]) > buffer_size:
                classification_buffer[cls_id].pop(0)

            # Smooth misclassification
            if len(classification_buffer[cls_id]) == buffer_size:
                if classification_buffer[cls_id].count(0) < buffer_size - 1:
                    # Correct to the majority class
                    frame_classifications[cls_id] = [(int(px), int(py)) for px, py in frame_classifications[cls_id]]

        for cls_id, trail in player_trails.items():
            if len(trail) > 1:
                for i in range(len(trail) - 1):
                    cv2.line(overlay_map, trail[i], trail[i + 1], dot_color, 2)

        for cls_id in [0, 1]:
            if not current_players[cls_id] and frame_no - player_last_seen[cls_id] > max_inactive_frames:
                player_trails[cls_id].clear()

    overlay_resized = cv2.resize(overlay_map, overlay_map_size)
    oh, ow = overlay_resized.shape[:2]
    if overlay_x + ow > frame_w:
        ow = frame_w - overlay_x
        overlay_resized = cv2.resize(overlay_resized, (ow, oh))
    if overlay_y + oh > frame_h:
        oh = frame_h - overlay_y
        overlay_resized = cv2.resize(overlay_resized, (ow, oh))

    roi = frame[overlay_y:overlay_y+oh, overlay_x:overlay_x+ow]
    alpha = 0.8
    blended = cv2.addWeighted(roi, 1 - alpha, overlay_resized, alpha, 0)
    frame[overlay_y:overlay_y+oh, overlay_x:overlay_x+ow] = blended

    out.write(frame)
    last_frame = frame.copy()
    if frame_no == 100:
        cv2_imshow(last_frame)
    elif frame_no == 400:
        cv2_imshow(frame)
    elif frame_no == 900:
        cv2_imshow(frame)

cap.release()
out.release()
print('Done! Video saved at:', output_video_path)


Output hidden; open in https://colab.research.google.com to view.